In [3]:
!pip install beautifulsoup4 requests pandas


     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ---------------------------------------- 59.7/59.7 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   ---------------------------------------- 147.9/147.9 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.9 kB ? eta -:--:--
   ---------------------------------------- 64.9/64.9 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.2/11.6 MB ? eta -:--:--
   -- ------------------------------------- 0.6/11.6 MB 7.6 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/11.6 MB 8.0 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/11.6 MB 9.3 MB/s eta 0:00:02
   ------- -------------------------------- 2.2/11.6 MB 10.2 MB/s eta 0:00:01
   --------- ------------------------------ 2.9/11.6 MB 10.7 MB/s eta 0:00:01
   ----------- -------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Dario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
from bs4 import BeautifulSoup
import sys
import requests
import pandas as pd

def get_page(url):
    headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/117.0"}

    try:
        page = requests.get(url, headers=headers)
        return page
    except Exception as e:
       # error_type, error_obj, error_info = sys.exc_info()
        print('ERROR FOR LINK:', url)
      #  print(error_type, 'Line:', error_info.tb_lineno)
        return None

base_url = 'http://www.panorama.com.al/'

# Generate Year-Month-Day Range for July to December 2018
links_jul_dec_2018 = []
for month in range(7, 13):  # July (7) to December (12)
    # Adjust days depending on the month
    if month in [7, 8, 10, 12]:  # Months with 31 days
        days_in_month = 31
    elif month in [9, 11]:  # Months with 30 days
        days_in_month = 30
    
    # Generate day URLs
    days = [f'2018/{str(month).zfill(2)}/{str(day).zfill(2)}' for day in range(1, days_in_month + 1)]
    links_jul_dec_2018.extend([base_url + day + '/' for day in days])

# Combine all the lists
links_list = links_jul_dec_2018

print("list:", links_list)


final_domains = []
final_links = []
final_titles = []
final_summaries = []
final_articles = []

for link in links_list:
    page = get_page(link)
    print("page: ", page)
    if page is None:
        continue

    soup = BeautifulSoup(page.text, "html.parser")

    domain_tag = soup.find('h1', class_='entry-title td-page-title')
    domain = domain_tag.text.strip() if domain_tag else 'Unknown'

    print("DOMAIN")
    print(domain)

    content1 = soup.find_all('h1', class_='entry-title td-module-title')
    titles = []
    links = []

    for c1 in content1:
        title_tag = c1.find('a', href=True)
        if title_tag:
            titles.append(title_tag.text)
            links.append(title_tag['href'])

    domain_list = [domain] * len(links)

    print("Titles:", titles)
    print("Links:", links)

    summaries = []
    articles = []

    for l in links:
        page = get_page(l)
        if page is None:
            summaries.append('')
            articles.append('')
            continue

        soup = BeautifulSoup(page.text, "html.parser")
        news = soup.find_all('p')
        if news:
            summary = news[0].text
            summaries.append(summary)
            article = ''.join([p.text for p in news[1:]])
            articles.append(article)
        else:
            summaries.append('')
            articles.append('')

    final_domains.extend(domain_list)
    final_links.extend(links)
    final_titles.extend(titles)
    final_summaries.extend(summaries)
    final_articles.extend(articles)

print(len(final_domains))
print(len(final_links))
print(len(final_titles))
print(len(final_summaries))
print(len(final_articles))

df = pd.DataFrame({
    'Domain': final_domains,
    'URL': final_links,
    'Title': final_titles,
    'Summary': final_summaries,
    'Article': final_articles
})

df.to_csv('news_panorama_jul_dec_2018.csv', index=False, encoding='utf-8')


list: ['http://www.panorama.com.al/2018/07/01/', 'http://www.panorama.com.al/2018/07/02/', 'http://www.panorama.com.al/2018/07/03/', 'http://www.panorama.com.al/2018/07/04/', 'http://www.panorama.com.al/2018/07/05/', 'http://www.panorama.com.al/2018/07/06/', 'http://www.panorama.com.al/2018/07/07/', 'http://www.panorama.com.al/2018/07/08/', 'http://www.panorama.com.al/2018/07/09/', 'http://www.panorama.com.al/2018/07/10/', 'http://www.panorama.com.al/2018/07/11/', 'http://www.panorama.com.al/2018/07/12/', 'http://www.panorama.com.al/2018/07/13/', 'http://www.panorama.com.al/2018/07/14/', 'http://www.panorama.com.al/2018/07/15/', 'http://www.panorama.com.al/2018/07/16/', 'http://www.panorama.com.al/2018/07/17/', 'http://www.panorama.com.al/2018/07/18/', 'http://www.panorama.com.al/2018/07/19/', 'http://www.panorama.com.al/2018/07/20/', 'http://www.panorama.com.al/2018/07/21/', 'http://www.panorama.com.al/2018/07/22/', 'http://www.panorama.com.al/2018/07/23/', 'http://www.panorama.com.al